##### Расчет параметра OVERDUE_FLG наличия просрочки по договору на определенную дату для раcчета фичи contract_ovd_cnt - количество просроченных договоров на дату принятия решения

In [1]:
pre = '../../' # путь до главной директории проекта

%run ../init.ipynb #подгрузить нужные библиотеки
%run ../../params.ipynb #параметры как логин пароль к БИ, параметры расчёта ( в нашем случае их нет)

In [2]:
# сюда таблицы служебные будем кидать, для отчетов
name = pre + "data/interm/auxilary/feature_contract_ovd_cnt.xlsx"
writer = ExcelWriter(name)

In [3]:
name = pre + "data/interm/target_data.pkl"
target_data = pd.read_pickle(name, compression={'method':'zip'})

print(len(target_data))
# target_data.head(1)

127968


In [4]:
def read_overdue_contracts():
    sql_query = """ 
                   SELECT DISTINCT CONTRS.INN, CONTRS.DECISION_DATE, CONTRS.CONTRACT_KEY, CONTRS.CONTRACT_APP_KEY ,
                    CASE WHEN OVD.CONTRACT_KEY    IS NOT NULL  AND OVD.CONTRACT_APP_KEY IS NOT NULL  THEN 1  ELSE 0  END AS OVERDUE_FLG
                  FROM PRE_CONTRACTS CONTRS
                  LEFT JOIN
                  ( SELECT DISTINCT PDD.SNAPSHOT_DT, PDD.CONTRACT_KEY, PDD.CONTRACT_APP_KEY, PDD.CBC_DESC, PDD.PAYMENT_NUM, PDD.PLAN_PAY_DT_ORIG, PDD.PLAN_AMT, PDD.PAYMENT_ITEM_KEY, PDD.CURRENCY_KEY, PDD.OFF_SCHEDULE, 
                              FDD.MAX_PAY_DT_ORIG, 
                              FDD.FACT_PAY_AMT
                            FROM DM.DM_DETAILS_DAILY PDD
                          LEFT JOIN
                            (SELECT DISTINCT PBD.CONTRACT_KEY, PBD.CONTRACT_APP_KEY, CBC_DESC, PAYMENT_NUM, PLAN_PAY_DT_ORIG, PLAN_AMT, PAYMENT_ITEM_KEY, CURRENCY_KEY, OFF_SCHEDULE,
                              MAX(PBD.PAY_DT_ORIG) OVER (PARTITION BY PBD.CONTRACT_KEY, PBD.CONTRACT_APP_KEY, CBC_DESC, PAYMENT_NUM, PLAN_PAY_DT_ORIG, PLAN_AMT, PAYMENT_ITEM_KEY, CURRENCY_KEY, OFF_SCHEDULE) AS MAX_PAY_DT_ORIG, 
                              SUM(NVL(PBD.FACT_PAY_AMT,0)) OVER (PARTITION BY PBD.CONTRACT_KEY, PBD.CONTRACT_APP_KEY, CBC_DESC, PAYMENT_NUM, PLAN_PAY_DT_ORIG, PLAN_AMT, PAYMENT_ITEM_KEY, CURRENCY_KEY, OFF_SCHEDULE) AS FACT_PAY_AMT
                            FROM
                              (SELECT CONTRACT_KEY, CONTRACT_APP_KEY, CBC_DESC, PAYMENT_NUM, PLAN_PAY_DT_ORIG, PLAN_AMT, PAY_DT_ORIG, FACT_PAY_AMT, PAYMENT_ITEM_KEY, CURRENCY_KEY, OFF_SCHEDULE
                              FROM DM.DM_DETAILS_DAILY
                              WHERE PAYMENT_ITEM_KEY  IN (9, 1000)
                              AND NVL(PLAN_AMT, 0)     > 0
                              AND PRE_PAY              > 0
                              ) PBD
                            INNER JOIN
                              ( SELECT * FROM PRE_CONTRACTS
                              ) PRE_CON
                            ON PBD.CONTRACT_KEY = PRE_CON.CONTRACT_KEY
                            AND PBD.CONTRACT_APP_KEY = PRE_CON.CONTRACT_APP_KEY
                            AND PBD.PAY_DT_ORIG < PRE_CON.DECISION_DATE - 1
                            ) FDD
                          ON PDD.CONTRACT_KEY                             = FDD.CONTRACT_KEY
                          AND PDD.CONTRACT_APP_KEY                          = FDD.CONTRACT_APP_KEY
                          AND PDD.CBC_DESC                                  = FDD.CBC_DESC
                          AND PDD.PAYMENT_NUM                               = FDD.PAYMENT_NUM
                          AND PDD.PLAN_PAY_DT_ORIG                          = FDD.PLAN_PAY_DT_ORIG
                          AND PDD.PLAN_AMT                                  = FDD.PLAN_AMT
                          AND PDD.PAYMENT_ITEM_KEY                          = FDD.PAYMENT_ITEM_KEY
                          AND PDD.CURRENCY_KEY                              = FDD.CURRENCY_KEY
                          AND PDD.OFF_SCHEDULE                              = FDD.OFF_SCHEDULE
                        WHERE  PDD.PAYMENT_ITEM_KEY  IN (9, 1000)
                              AND NVL(PDD.PLAN_AMT, 0)     > 0
                              AND PDD.PRE_PAY              > 0 
                        AND (PDD.PLAN_AMT - NVL(FDD.FACT_PAY_AMT, 0) >0
                        OR  (PDD.PLAN_PAY_DT_ORIG < NVL(FDD.MAX_PAY_DT_ORIG,to_date ('01.01.2400', 'dd.mm.yyyy'))
                        AND (EXTRACT (MONTH FROM PDD.PLAN_PAY_DT_ORIG) <> EXTRACT (MONTH FROM NVL(FDD.MAX_PAY_DT_ORIG,to_date ('01.01.2400', 'dd.mm.yyyy')))
                        OR EXTRACT (YEAR FROM PDD.PLAN_PAY_DT_ORIG)      <> EXTRACT (YEAR FROM NVL(FDD.MAX_PAY_DT_ORIG,to_date ('01.01.2400', 'dd.mm.yyyy'))))))
                      ) OVD
                    ON OVD.CONTRACT_KEY = CONTRS.CONTRACT_KEY
                    AND OVD.CONTRACT_APP_KEY = CONTRS.CONTRACT_APP_KEY
                    AND OVD.PLAN_PAY_DT_ORIG < CONTRS.DECISION_DATE - 1
                """
                
    df = read_from_oracle(sql_query, title='overdue_contracts')
    
    return df

overdue_contracts = read_overdue_contracts()
# overdue_contracts.head(1)

Размер считанных данных overdue_contracts - (207598, 5)


In [5]:
overdue_contracts.rename(columns={'DECISION_DATE':'decision_date'}, inplace = True)
overdue_contracts['CONTRACT_KEY'] = overdue_contracts['CONTRACT_KEY'].astype(str)

In [6]:
overdue_contracts.dtypes

INN                         object
decision_date       datetime64[ns]
CONTRACT_KEY                object
CONTRACT_APP_KEY             int64
OVERDUE_FLG                  int64
dtype: object

In [7]:
overdue_contracts.to_pickle(pre+"data/interm/overdue_contracts.pkl", compression={'method': 'zip'})

In [8]:
# сохраняем tmp таблицы
if len(writer.sheets):
    writer.save() 